<a href="https://colab.research.google.com/github/salathegroup/multi-lang-vaccine-sentiment/blob/master/Multilingual_Vaccine_Categorisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Multilingual Vaccine Categorisation
Performing vaccine stance fintetuning/categorisation training on various multilingual dataset



# Step 1

In [0]:
#@markdown ##Autenticate, Import Libraries and Copy Data Files

#import modules
import sys, os, json, csv, datetime
from google.colab import auth
from google.colab import drive
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import Progbar

#Authenticate
auth.authenticate_user()

##Copy all the training data locally
if not os.path.exists('data'):
  os.makedirs('data')
  !gsutil -m cp -r gs://perepublic/EPFL_multilang/data/ /content/
else:
  print('All training files has already been copied to /content/data')

#Clone Bert
if not os.path.exists('bert_repo'):  
  !test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
else:
  print('The Bert repository has already been cloned')

if not '/content/bert_repo' in sys.path:
  sys.path += ['bert_repo']
  sys.path += ['/content/bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

#Mount Google Drive   
if not os.path.exists('/content/GDrive/'):
  print('Mounting Google Drive')
  drive.mount('/content/GDrive', force_remount=False)
else:
  print('Your Google Drive is already mounted at /content/GDrive')

#Enable TPUs
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

# Upload credentials to TPU for all future sessions on this TPU.
with tf.Session(TPU_ADDRESS) as session:
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

#Define some custom functions
class vaccineStanceProcessor(run_classifier.DataProcessor):
  """Processor for the NoRec data set."""

  def get_train_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, 'train.tsv')), 'train')

  def get_dev_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, 'dev.tsv')), 'dev')

  def get_test_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, 'test.tsv')), 'test')

  def get_labels(self):
    """See base class."""
    return ['positive','neutral','negative']

  def _create_examples(self, lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(lines):
      # Only the test set has a header
      if set_type == 'test' and i == 0:
        continue
      guid = '%s-%s' % (set_type, i)
      if set_type == 'test':
        text_a = tokenization.convert_to_unicode(line[3])
        #Set a dummy value. This is not used
        label = 'positive'
      else:
        text_a = tokenization.convert_to_unicode(line[3])
        label = tokenization.convert_to_unicode(line[1])
      examples.append(
          run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Train the model
def model_train(estimator):
  # Time to train another model - Clean the output directory.
  !gsutil rm -r $TEMP_OUTPUT_DIR

  # Force TF Hub writes to the GS bucket we provide.
  os.environ['TFHUB_CACHE_DIR'] = TEMP_OUTPUT_DIR
  
  print('Fine tuning BERT base model normally takes a few minutes. Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  print('  Train steps = {}'.format(num_train_steps))
  print('  Epochs = {}'.format(NUM_TRAIN_EPOCHS))
  
  tf.logging.info('  Num steps = %d', num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))

# Evaluate the model.
def model_eval(estimator):
  #Copy the vocab and config files from the base directory to the temporary finetune directory so that model can be restored 
  vocabfile = os.path.join(BERT_MODEL_DIR,'vocab.txt')
  configfile = os.path.join(BERT_MODEL_DIR,'bert_config.json')
  !gsutil cp $vocabfile $TEMP_OUTPUT_DIR/
  !gsutil cp $configfile $TEMP_OUTPUT_DIR/
  
  eval_examples = processor.get_dev_examples(EVAL_ANNOT_DATASET_DIR)
  print("there")
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(TEMP_OUTPUT_DIR, 'eval_results.txt')
  with tf.gfile.GFile(output_eval_file, 'w') as writer:
    print('***** Eval results *****')
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write('%s = %s\n' % (key, str(result[key])))

  #Write log to Training Log File

  with open(TRAINING_LOG_FILE, mode='a+') as output:
    output_writer = csv.writer(output, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    output_writer.writerow([format(datetime.datetime.now()),USERNAME,BERT_MODEL_FILE, TRAIN_ANNOT_DATASET, EVAL_ANNOT_DATASET,NUM_TRAIN_EPOCHS, LEARNING_RATE,MAX_SEQ_LENGTH,result['eval_accuracy'], "Not Calulated Yet",result['eval_loss'], result['loss'], COMMENT])

def model_predict(estimator):
  # Make predictions on a subset of eval examples
  prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)

  for example, prediction in zip(prediction_examples, predictions):
    print('text_a: %s\nlabel:%s\nprediction:%s\n' % (example.text_a, str(example.label), prediction['probabilities']))

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))



# Step 2

In [0]:
#@markdown ##Initiate Model and Parameters
#@markdown Username and Comment are only used in temporal folders and in the train log file

#General Files and Settings
USERNAME = "pere" #@param {type:"string"}
#@markdown <br />

COMMENT = 'First test of training model'#@param {type:"string"}
#@markdown <br />
BERT_MODEL_DIR = 'gs://perepublic/transformer_eval/multi_cased_L-12_H-768_A-12/'#@param {type:"string"}
BERT_MODEL_NAME = 'model.ckpt.index'#@param {type:"string"}
TEMP_OUTPUT_DIR = 'gs://perepublic/finetuned_models/' #@param {type:"string"}
TRAINING_LOG_FILE = '/content/GDrive/My Drive/EPFL_vaccine_customloss/trainlog.csv'#@param {type:"string"}
#@markdown <br />
TRAIN_ANNOT_DATASET = 'cb-annot-en' #@param ['cb-annot-en','cb-annot-en-de','cb-annot-en-es','cb-annot-en-fr','cb-annot-en-pt','cb-annot-en-sm','cb-annot-en-de-sm','cb-annot-en-es-sm','cb-annot-en-fr-sm','cb-annot-en-pt-sm']
EVAL_ANNOT_DATASET = 'cb-annot-en' #@param ['cb-annot-en','cb-annot-en-de','cb-annot-en-es','cb-annot-en-fr','cb-annot-en-pt','cb-annot-en-sm','cb-annot-en-de-sm','cb-annot-en-es-sm','cb-annot-en-fr-sm','cb-annot-en-pt-sm']

#Complete some paths
BERT_MODEL_FILE = os.path.join(BERT_MODEL_DIR,BERT_MODEL_NAME)
TRAIN_ANNOT_DATASET_DIR = os.path.join('/content/data',TRAIN_ANNOT_DATASET)
EVAL_ANNOT_DATASET_DIR = os.path.join('/content/data',EVAL_ANNOT_DATASET)


#Model Finetuning Hyperparameters
NUM_TRAIN_EPOCHS = 3
LEARNING_RATE = 2e-5
MAX_SEQ_LENGTH = 128
TRAIN_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 64
WARMUP_PROPORTION = 0.1

#Other Config
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500
USE_TPU = True
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000
LOWER_CASED = False

#Do checks to see if all necessary files exists
if not tf.gfile.Exists(BERT_MODEL_DIR):
  print('Can not access the Bert model directory')
  sys.exit('Stopping execution!')

if not tf.gfile.Exists(TRAIN_ANNOT_DATASET_DIR):
  print('Can not access the training files')
  sys.exit('Stopping execution!')

if not tf.gfile.Exists(EVAL_ANNOT_DATASET_DIR):
  print('Can not access the training files')
  sys.exit('Stopping execution!')

if not tf.gfile.Exists(TEMP_OUTPUT_DIR):
  print('Can not access the temporary directory for storing finetuned models')
  sys.exit('Stopping execution!')
else:
  TEMP_OUTPUT_DIR += USERNAME

if not tf.gfile.Exists(os.path.join(BERT_MODEL_DIR,'vocab.txt')):
  print('Can not access the Bert model vocabulary file. This file should be located in the Bert model dir')
  sys.exit('Stopping execution!')

if not tf.gfile.Exists(os.path.join(BERT_MODEL_DIR,'bert_config.json')):
  print('Can not access the Bert model config file. This file should be located in the Bert model dir')
  sys.exit('Stopping execution!')

if not tf.gfile.Exists(BERT_MODEL_FILE):
  print('Can not access the Bert model file')
  sys.exit('Stopping execution!')

if not tf.gfile.Exists(TRAINING_LOG_FILE):
  print('The Training Log File used to write training results is not available. Creating one for you.')
  f = open(TRAINING_LOG_FILE, 'w+')
  head = 'Date,User,Model,Train_Annot_Dataset,Eval_Annot_Dataset,Num_Train_Epochs,Learning_Rate,Max_Seq_Length,Eval_Accuracy,Eval_F_score,Eval_Loss,Loss,Comment\n'
  f.write(head)
  f.close()
  print('An empty Training Log File has been created in '+TRAINING_LOG_FILE)

print ('All necessary files exist!')

#Initiation
tokenizer = tokenization.FullTokenizer(vocab_file=os.path.join(BERT_MODEL_DIR,'vocab.txt'),do_lower_case=LOWER_CASED)
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
processor = vaccineStanceProcessor()

label_list = processor.get_labels()

# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TRAIN_ANNOT_DATASET_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
print('There are a total of '+str(len(train_examples))+' training examples in '+TRAIN_ANNOT_DATASET_DIR+'.\nWe will be training for '+str(NUM_TRAIN_EPOCHS)+' epochs, which means '+str(num_train_steps)+' training steps with a batch size of '+str(TRAIN_BATCH_SIZE)+'.\n\n')

model_fn = run_classifier.model_fn_builder(
  bert_config=modeling.BertConfig.from_json_file(os.path.join(BERT_MODEL_DIR,'bert_config.json')),
  num_labels=len(label_list),
  init_checkpoint=BERT_MODEL_FILE,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  use_one_hot_embeddings=True
)

estimator_from_checkpoints = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(TEMP_OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)
print ('Model is initiated and ready to train!')


# Step 3

In [0]:
#Training and evaluation
model_train(estimator_from_checkpoints)
model_eval(estimator_from_checkpoints)

Removing gs://perepublic/finetuned_models/pere/events.out.tfevents.1580820790.a292414309a6#1580823478664569...
Removing gs://perepublic/finetuned_models/pere/graph.pbtxt#1580823474771478...
Removing gs://perepublic/finetuned_models/pere/model.ckpt-0.data-00000-of-00001#1580823515508426...
Removing gs://perepublic/finetuned_models/pere/model.ckpt-0.index#1580823516363402...
/ [4 objects]                                                                   
Operation completed over 4 objects.                                              
Fine tuning BERT base model normally takes a few minutes. Please wait...

INFO:tensorflow:Writing example 0 of 6000
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train-0
INFO:tensorflow:tokens: [CLS] " " " Living Good . . Feeling Way Better . . . Me & am ##p ; < @ user > Turn ##ing Up Last Night " " # GM ##F x # MM ##R @ Baru Lounge < url > " [SEP]
INFO:tensorflow:input_ids: 101 107 107 107 24371 13073 119 119 68306 15661 34961 119 119 119 11589 111